In [34]:
import subprocess
 
default_properies = (
  "timestamp",
  "gpu_name",
  #"gpu_uuid",
  "index",
  "memory.total",
  "memory.used",
  "memory.free",
  "utilization.gpu",
  "utilization.memory",
)
 
def get_gpu_properties(
  cmd_path="C:\Windows\System32\\nvidia-smi.exe",
  target_properties=default_properies,
  noheader=True,
  nounits=True
  ):
  """
  CUDA GPUのプロパティ情報取得
 
  Parameters
  ----------
  cmd_path : str
    コマンドラインから"nvidia-smi"を実行する際のパス
  target_properties : obj
    取得するプロパティ情報
    プロパティ情報の詳細は"nvidia-smi --help-query-gpu"で取得可能
  noheader : bool
    skip the first line with column headers
  nounits : bool
    don't print units for numerical values
 
  Returns
  -------
  gpu_properties : list
    gpuごとのproperty情報
  """
    
  # formatオプション定義
  format_option = "--format=csv"
  if noheader:
      format_option += ",noheader"
  if nounits:
      format_option += ",nounits"
 
  # コマンド生成
  cmd = '%s --query-gpu=%s %s' % (cmd_path, ','.join(target_properties), format_option)
 
  # サブプロセスでコマンド実行
  cmd_res = subprocess.check_output(cmd, shell=True)
    
  # コマンド実行結果をオブジェクトに変換
  gpu_lines = cmd_res.decode().split('\n')
  # リストの最後の要素に空行が入るため除去
  gpu_lines = [ line.strip() for line in gpu_lines if line.strip() != '' ]
 
  # ", "ごとにプロパティ情報が入っているのでdictにして格納
  gpu_properties = [ { k: v for k, v in zip(target_properties, line.split(', ')) } for line in gpu_lines ]
 
  return gpu_properties

In [35]:
properties = get_gpu_properties()
# GPU Property情報出力
for property in properties:
  print(property)
  print(property["gpu_name"], "[", property["index"], "] used", property["memory.used"], "MiB" )
  # {'timestamp': '2021/10/04 12:40:37.100', 'gpu_name': 'GeForce GTX 1080 Ti', 'index': '1', 'memory.total': '11178', 'memory.used': '2', 'memorfree': '11176', 'utilization.gpu': '0', 'utilization.memory': '0'}
  # GeForce GTX 1080 Ti [ 0 ] used 19 MiB

{'timestamp': '2022/10/19 10:38:53.063', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '41', 'utilization.memory': '11'}
NVIDIA RTX A5000 Laptop GPU [ 0 ] used 228 MiB


In [36]:
import threading
import time
 
g_IsMonitor = False
g_GpuPropertiesList = []
 
def monitoring_gpu_task(interval):
  global g_IsMonitor
  global g_GpuPropertiesList
 
  while g_IsMonitor:
    time.sleep(interval)
    res = get_gpu_properties()
    g_GpuPropertiesList.append( res )
 
def start_gpu_monitor(interval):
  global g_IsMonitor
  global g_GpuPropertiesList
 
  g_IsMonitor = True
  g_GpuPropertiesList.clear()
 
  monitor_thread = threading.Thread(target=monitoring_gpu_task, args=(interval, ))
  monitor_thread.start()
 
def end_gpu_monitor():
  global g_IsMonitor
  global g_GpuPropertiesList
  g_IsMonitor = False
  return g_GpuPropertiesList
 
# 1sec間隔でGPUのProperty情報を取得
for number in range(3):
  start_gpu_monitor(1)
  time.sleep(3)
  properties_list = end_gpu_monitor()
  print(properties_list)
# [
#   [
#     {'timestamp': '2021/10/04 15:33:24.015', 'gpu_name': 'GeForce GTX 1080 Ti', 'index': '0', 'memory.total': '11178', 'memory.used': '19', 'memorfree': '11159', 'utilization.gpu': '0', 'utilization.memory': '0'}, 
#     {'timestamp': '2021/10/04 15:33:24.016', 'gpu_name': 'GeForce GTX 1080 Ti', 'index': '1', 'memory.total': '11178', 'memory.used': '2', 'memorfree': '11176', 'utilization.gpu': '0', 'utilization.memory': '0'}
#   ], 
#   [
#     {'timestamp': '2021/10/04 15:33:25.039', 'gpu_name': 'GeForce GTX 1080 Ti', 'index': '0', 'memory.total': '11178', 'memory.used': '19', 'memorfree': '11159', 'utilization.gpu': '0', 'utilization.memory': '0'}, 
#     {'timestamp': '2021/10/04 15:33:25.039', 'gpu_name': 'GeForce GTX 1080 Ti', 'index': '1', 'memory.total': '11178', 'memory.used': '2', 'memorfree': '11176', 'utilization.gpu': '0', 'utilization.memory': '0'}
#   ]
# ]
 

[[{'timestamp': '2022/10/19 10:38:54.153', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '43', 'utilization.memory': '14'}], [{'timestamp': '2022/10/19 10:38:55.227', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '31', 'utilization.memory': '13'}]]
[[{'timestamp': '2022/10/19 10:38:56.286', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '23', 'utilization.memory': '6'}], [{'timestamp': '2022/10/19 10:38:57.152', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '38', 'utilization.memory': '11'}], [{'timestamp': '2022/10/19 10:38:57.342', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 

In [41]:
properties_list[0],properties_list[1]
print(properties_list[0])
type(properties_list[0][0])
properties_list[0][0]['timestamp']
for dict in properties_list:
  dict[0]['timestamp'] += ('"')

[{'timestamp': '2022/10/19 10:38:59.256', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '38', 'utilization.memory': '10'}]


In [42]:
import csv
outputlist = []

with open('sample_gpu.csv', 'w', newline="") as f:
  writer = csv.DictWriter(f, ['timestamp','memory.used','memory.free', 'index', 'utilization.gpu', 'utilization.memory', 'gpu_name', 'memory.total'])
  writer.writeheader()
  for dict in properties_list:
    writer.writerow(dict[0])
    print(dict[0])

{'timestamp': '2022/10/19 10:38:59.256"', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '38', 'utilization.memory': '10'}
{'timestamp': '2022/10/19 10:38:59.461"', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '38', 'utilization.memory': '10'}
{'timestamp': '2022/10/19 10:39:00.150"', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '38', 'utilization.memory': '10'}
{'timestamp': '2022/10/19 10:39:00.323"', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.total': '16384', 'memory.used': '228', 'memory.free': '15972', 'utilization.gpu': '38', 'utilization.memory': '10'}
{'timestamp': '2022/10/19 10:39:00.526"', 'gpu_name': 'NVIDIA RTX A5000 Laptop GPU', 'index': '0', 'memory.t